In [3]:
import sys
import pathlib
import numpy as np
import pandas as pd
sys.path.insert(0, "../../scripts")
from utils import load_data


from pycytominer.cyto_utils import infer_cp_features


import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from sklearn.decomposition import PCA
from tensorflow import keras

from vae import VAE

from tensorflow.keras.models import Model, Sequential
import seaborn
import random as python_random
import tensorflow as tf
# import umap

In [4]:
data_splits = ["train", "test", "valid", "complete"]
data_dict = load_data(data_splits)

In [5]:
# Prepare data for training
meta_features = infer_cp_features(data_dict["train"], metadata=True)
cp_features = infer_cp_features(data_dict["train"])

train_features_df = data_dict["train"].reindex(cp_features, axis="columns")
train_meta_df = data_dict["train"].reindex(meta_features, axis="columns")

test_features_df = data_dict["test"].reindex(cp_features, axis="columns")
test_meta_df = data_dict["test"].reindex(meta_features, axis="columns")

valid_features_df = data_dict["valid"].reindex(cp_features, axis="columns")
valid_meta_df = data_dict["valid"].reindex(meta_features, axis="columns")

complete_features_df = data_dict["complete"].reindex(cp_features, axis="columns")
complete_meta_df = data_dict["complete"].reindex(meta_features, axis="columns")


In [6]:
print(train_features_df.shape)
train_features_df.head(3)

(8294, 588)


,Cells_AreaShape_FormFactor,Cells_AreaShape_Orientation,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_5_1,Cells_AreaShape_Zernike_6_2,Cells_AreaShape_Zernike_7_1,Cells_AreaShape_Zernike_7_7,Cells_AreaShape_Zernike_9_3,Cells_Correlation_Correlation_DNA_AGP,...,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumEntropy_AGP_10_0,Nuclei_Texture_SumEntropy_AGP_20_0,Nuclei_Texture_SumEntropy_AGP_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_Mito_5_0,Nuclei_Texture_SumVariance_DNA_5_0,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_DNA_5_0
0,0.56071,0.63708,0.57961,0.51030,0.61695,0.30869,0.43981,0.79826,0.72477,0.68589,...,0.17858,0.29336,0.26447,0.30197,0.64786,0.49525,0.53512,0.40495,0.46056,0.43647
1,0.59702,0.55453,0.62561,0.47959,0.59022,0.33269,0.49875,0.88042,0.67159,0.61703,...,0.09030,0.15088,0.13334,0.14052,0.59229,0.49294,0.43221,0.33969,0.39208,0.36205
2,0.64623,0.59825,0.62822,0.52100,0.57452,0.26367,0.48172,0.86322,0.67588,0.69732,...,0.18125,0.23244,0.20723,0.24062,0.54642,0.43920,0.41183,0.30375,0.35385,0.33266


In [7]:
def shuffle_each_column(df):
    columns = df.columns
    df_copy = df.copy()
    for column in columns:
        df_copy[column] = df_copy[column].sample(frac=1).reset_index(drop=True)
    return (df_copy)

In [8]:
train_features_df = shuffle_each_column(train_features_df)
valid_features_df = shuffle_each_column(valid_features_df)

In [9]:
encoder_architecture=[250]
decoder_architecture=[250]

In [10]:
cp_vae = VAE(
    input_dim=train_features_df.shape[1],
    latent_dim=10,
    batch_size=96,
    encoder_batch_norm=True,
    epochs=100,
    learning_rate=0.001,
    encoder_architecture=encoder_architecture,
    decoder_architecture=decoder_architecture,
    beta=.3,
    verbose=True,
)
cp_vae.compile_vae()

2022-01-14 01:44:28.944703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-14 01:44:28.944743: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-14 01:44:28.944771: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-3-81): /proc/driver/nvidia/version does not exist
2022-01-14 01:44:28.945046: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/miniconda3/envs/cell-painting-vae/lib/python3.7/site-packages/keras/optimizer_v

In [11]:
cp_vae.train(x_train=train_features_df, x_test=valid_features_df)

2022-01-14 01:44:30.529914: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
87/87 [==============================] - 4s 13ms/step - loss: 33.4196 - recon: 32.0597 - kl: 1.3599 - mmd: 0.0000e+00 - val_loss: 11.5276 - val_recon: 10.6055 - val_kl: 0.9221 - val_mmd: 0.0000e+00
Epoch 2/100
87/87 [==============================] - 1s 7ms/step - loss: 11.5415 - recon: 10.3365 - kl: 1.2050 - mmd: 0.0000e+00 - val_loss: 10.7611 - val_recon: 9.4479 - val_kl: 1.3132 - val_mmd: 0.0000e+00
Epoch 3/100
87/87 [==============================] - 1s 7ms/step - loss: 8.9388 - recon: 7.8849 - kl: 1.0539 - mmd: 0.0000e+00 - val_loss: 14.3468 - val_recon: 11.6339 - val_kl: 2.7128 - val_mmd: 0.0000e+00
Epoch 4/100
87/87 [==============================] - 1s 7ms/step - loss: 7.4698 - recon: 6.5535 - kl: 0.9164 - mmd: 0.0000e+00 - val_loss: 8.1009 - val_recon: 6.6402 - val_kl: 1.4607 - val_mmd: 0.0000e+00
Epoch 5/100
87/87 [==============================] - 1s 7ms/step - loss: 6.6829 - recon: 5.8854 - kl: 0.7975 - mmd: 0.0000e+00 - val_loss: 7.1492 - val_recon: 5.8976 - va

Epoch 41/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3488 - recon: 5.3079 - kl: 0.0409 - mmd: 0.0000e+00 - val_loss: 5.2363 - val_recon: 5.1888 - val_kl: 0.0475 - val_mmd: 0.0000e+00
Epoch 42/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3451 - recon: 5.3061 - kl: 0.0390 - mmd: 0.0000e+00 - val_loss: 5.2301 - val_recon: 5.1846 - val_kl: 0.0455 - val_mmd: 0.0000e+00
Epoch 43/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3447 - recon: 5.3074 - kl: 0.0373 - mmd: 0.0000e+00 - val_loss: 5.2279 - val_recon: 5.1845 - val_kl: 0.0434 - val_mmd: 0.0000e+00
Epoch 44/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3421 - recon: 5.3065 - kl: 0.0356 - mmd: 0.0000e+00 - val_loss: 5.2240 - val_recon: 5.1825 - val_kl: 0.0415 - val_mmd: 0.0000e+00
Epoch 45/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3393 - recon: 5.3052 - kl: 0.0341 - mmd: 0.0000e+00 - val_loss: 5.2185 - val_recon: 5.1791 - val_kl:

Epoch 81/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2987 - recon: 5.2951 - kl: 0.0036 - mmd: 0.0000e+00 - val_loss: 5.1765 - val_recon: 5.1726 - val_kl: 0.0038 - val_mmd: 0.0000e+00
Epoch 82/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2992 - recon: 5.2959 - kl: 0.0033 - mmd: 0.0000e+00 - val_loss: 5.1834 - val_recon: 5.1800 - val_kl: 0.0034 - val_mmd: 0.0000e+00
Epoch 83/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2964 - recon: 5.2935 - kl: 0.0029 - mmd: 0.0000e+00 - val_loss: 5.1788 - val_recon: 5.1758 - val_kl: 0.0030 - val_mmd: 0.0000e+00
Epoch 84/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2967 - recon: 5.2942 - kl: 0.0026 - mmd: 0.0000e+00 - val_loss: 5.1744 - val_recon: 5.1717 - val_kl: 0.0027 - val_mmd: 0.0000e+00
Epoch 85/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2952 - recon: 5.2930 - kl: 0.0023 - mmd: 0.0000e+00 - val_loss: 5.1746 - val_recon: 5.1723 - val_kl:

In [12]:
cp_vae.vae

In [13]:
encoder = cp_vae.encoder_block["encoder"]
decoder = cp_vae.decoder_block["decoder"]
encoder.save("models/level5EncoderShuffled_beta")
decoder.save("models/level5DecoderShuffled_beta")

2022-01-14 01:45:37.187666: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/level5EncoderShuffled_beta/assets
INFO:tensorflow:Assets written to: models/level5DecoderShuffled_beta/assets


In [14]:
# Save training performance
history_df = pd.DataFrame(cp_vae.vae.history.history)
history_df

,loss,recon,kl,mmd,val_loss,val_recon,val_kl,val_mmd
0,33.419617,32.059681,1.359945,0.0,11.527626,10.605546,0.922080,0.0
1,11.541502,10.336514,1.204988,0.0,10.761103,9.447875,1.313227,0.0
2,8.938849,7.884923,1.053928,0.0,14.346759,11.633946,2.712813,0.0
3,7.469841,6.553485,0.916354,0.0,8.100918,6.640206,1.460711,0.0
4,6.682912,5.885369,0.797541,0.0,7.149210,5.897563,1.251648,0.0
...,...,...,...,...,...,...,...,...
95,5.290754,5.290343,0.000409,0.0,5.172284,5.171879,0.000405,0.0
96,5.291967,5.291624,0.000343,0.0,5.164517,5.164182,0.000335,0.0
97,5.290632,5.290350,0.000284,0.0,5.169976,5.169702,0.000274,0.0
98,5.290751,5.290514,0.000234,0.0,5.166712,5.166485,0.000227,0.0


In [15]:
# history_df.to_csv('level5_training_random.csv')

In [16]:
# original_training_data  = pd.read_csv('level5_training.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'level5_training.csv'

In [ ]:
# plt.figure(figsize=(7, 5), dpi = 400)
# plt.plot(original_training_data["loss"], label="Training data")
# plt.plot(original_training_data["val_loss"], label="Validation data")
# plt.plot(history_df["loss"], label="Shuffled training data")
# plt.plot(history_df["val_loss"], label="Shuffled validation data")
# # plt.title("Loss for VAE training on Cell Painting Level 5 data")
# plt.ylabel("MSE + KL Divergence")
# plt.xlabel("No. Epoch")
# plt.legend()
# plt.show()

In [17]:
cp_vae = VAE(
    input_dim=train_features_df.shape[1],
    latent_dim=10,
    batch_size=96,
    encoder_batch_norm=True,
    epochs=100,
    learning_rate=0.001,
    encoder_architecture=encoder_architecture,
    decoder_architecture=decoder_architecture,
    beta=1,
    verbose=True,
)
cp_vae.compile_vae()

/home/ubuntu/miniconda3/envs/cell-painting-vae/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [18]:
cp_vae.train(x_train=train_features_df, x_test=valid_features_df)

Epoch 1/100
87/87 [==============================] - 3s 13ms/step - loss: 36.9487 - recon: 32.5821 - kl: 4.3666 - mmd: 0.0000e+00 - val_loss: 15.1795 - val_recon: 12.0833 - val_kl: 3.0962 - val_mmd: 0.0000e+00
Epoch 2/100
87/87 [==============================] - 1s 7ms/step - loss: 13.8791 - recon: 10.3486 - kl: 3.5305 - mmd: 0.0000e+00 - val_loss: 13.7657 - val_recon: 10.0466 - val_kl: 3.7191 - val_mmd: 0.0000e+00
Epoch 3/100
87/87 [==============================] - 1s 7ms/step - loss: 10.6387 - recon: 7.8289 - kl: 2.8098 - mmd: 0.0000e+00 - val_loss: 12.4224 - val_recon: 8.3344 - val_kl: 4.0880 - val_mmd: 0.0000e+00
Epoch 4/100
87/87 [==============================] - 1s 7ms/step - loss: 8.7534 - recon: 6.5319 - kl: 2.2216 - mmd: 0.0000e+00 - val_loss: 10.1248 - val_recon: 6.8207 - val_kl: 3.3041 - val_mmd: 0.0000e+00
Epoch 5/100
87/87 [==============================] - 1s 7ms/step - loss: 7.6622 - recon: 5.8918 - kl: 1.7704 - mmd: 0.0000e+00 - val_loss: 8.4447 - val_recon: 5.8090 - 

Epoch 41/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3676 - recon: 5.3085 - kl: 0.0591 - mmd: 0.0000e+00 - val_loss: 5.2534 - val_recon: 5.1867 - val_kl: 0.0666 - val_mmd: 0.0000e+00
Epoch 42/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3631 - recon: 5.3076 - kl: 0.0555 - mmd: 0.0000e+00 - val_loss: 5.2471 - val_recon: 5.1843 - val_kl: 0.0628 - val_mmd: 0.0000e+00
Epoch 43/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3594 - recon: 5.3074 - kl: 0.0520 - mmd: 0.0000e+00 - val_loss: 5.2438 - val_recon: 5.1853 - val_kl: 0.0585 - val_mmd: 0.0000e+00
Epoch 44/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3568 - recon: 5.3081 - kl: 0.0487 - mmd: 0.0000e+00 - val_loss: 5.2390 - val_recon: 5.1847 - val_kl: 0.0543 - val_mmd: 0.0000e+00
Epoch 45/100
87/87 [==============================] - 1s 7ms/step - loss: 5.3511 - recon: 5.3056 - kl: 0.0455 - mmd: 0.0000e+00 - val_loss: 5.2385 - val_recon: 5.1876 - val_kl:

87/87 [==============================] - 1s 7ms/step - loss: 5.2958 - recon: 5.2956 - kl: 2.4487e-04 - mmd: 0.0000e+00 - val_loss: 5.1737 - val_recon: 5.1735 - val_kl: 2.3205e-04 - val_mmd: 0.0000e+00
Epoch 81/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2956 - recon: 5.2954 - kl: 1.8635e-04 - mmd: 0.0000e+00 - val_loss: 5.1828 - val_recon: 5.1826 - val_kl: 1.7498e-04 - val_mmd: 0.0000e+00
Epoch 82/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2955 - recon: 5.2954 - kl: 1.4455e-04 - mmd: 0.0000e+00 - val_loss: 5.1729 - val_recon: 5.1728 - val_kl: 1.3339e-04 - val_mmd: 0.0000e+00
Epoch 83/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2935 - recon: 5.2933 - kl: 1.0804e-04 - mmd: 0.0000e+00 - val_loss: 5.1754 - val_recon: 5.1753 - val_kl: 9.9703e-05 - val_mmd: 0.0000e+00
Epoch 84/100
87/87 [==============================] - 1s 7ms/step - loss: 5.2936 - recon: 5.2935 - kl: 8.0908e-05 - mmd: 0.0000e+00 - val_loss: 5.1745 - val_

In [19]:
cp_vae.vae

In [20]:
# Save training performance
history_df = pd.DataFrame(cp_vae.vae.history.history)
history_df

,loss,recon,kl,mmd,val_loss,val_recon,val_kl,val_mmd
0,36.948734,32.582123,4.366626,0.0,15.179503,12.083273,3.096229,0.0
1,13.879109,10.348588,3.530521,0.0,13.765688,10.046631,3.719057,0.0
2,10.638667,7.828871,2.809797,0.0,12.422438,8.334407,4.088029,0.0
3,8.753426,6.531874,2.221552,0.0,10.124815,6.820738,3.304077,0.0
4,7.662220,5.891835,1.770385,0.0,8.444698,5.809000,2.635699,0.0
...,...,...,...,...,...,...,...,...
95,5.290924,5.290915,0.000010,0.0,5.166611,5.166602,0.000009,0.0
96,5.290392,5.290384,0.000008,0.0,5.173748,5.173740,0.000008,0.0
97,5.291130,5.291122,0.000008,0.0,5.169553,5.169545,0.000008,0.0
98,5.290127,5.290120,0.000008,0.0,5.165061,5.165055,0.000007,0.0


In [21]:
encoder = cp_vae.encoder_block["encoder"]
decoder = cp_vae.decoder_block["decoder"]
encoder.save("models/level5EncoderShuffled_vanilla")
decoder.save("models/level5DecoderShuffled_vanilla")

INFO:tensorflow:Assets written to: models/level5EncoderShuffled_vanilla/assets
INFO:tensorflow:Assets written to: models/level5DecoderShuffled_vanilla/assets


In [22]:
# history_df.to_csv('level5_training_vanilla_random.csv')

In [23]:
# original_training_data  = pd.read_csv('level5_training_vanilla.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'level5_training_vanilla.csv'

In [ ]:
# plt.figure(figsize=(7, 5), dpi = 400)
# plt.plot(original_training_data["loss"], label="Training data")
# plt.plot(original_training_data["val_loss"], label="Validation data")
# plt.plot(history_df["loss"], label="Shuffled training data")
# plt.plot(history_df["val_loss"], label="Shuffled validation data")
# # plt.title("Loss for VAE training on Cell Painting Level 5 data")
# plt.ylabel("MSE + KL Divergence")
# plt.xlabel("No. Epoch")
# plt.legend()
# plt.show()

In [24]:
cp_vae = VAE(
    input_dim=train_features_df.shape[1],
    latent_dim=10,
    batch_size=96,
    encoder_batch_norm=True,
    epochs=30,
    learning_rate=0.001,
    encoder_architecture=encoder_architecture,
    decoder_architecture=decoder_architecture,
    beta=0,
    lam=1000,
    verbose=True,
)
cp_vae.compile_vae()

/home/ubuntu/miniconda3/envs/cell-painting-vae/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [25]:
cp_vae.train(x_train=train_features_df, x_test=valid_features_df)

Epoch 1/30
87/87 [==============================] - 3s 13ms/step - loss: 53.2570 - recon: 33.4957 - kl: 0.0000e+00 - mmd: 20.1500 - val_loss: 55.2956 - val_recon: 10.6316 - val_kl: 0.0000e+00 - val_mmd: 42.8630
Epoch 2/30
87/87 [==============================] - 1s 7ms/step - loss: 23.4737 - recon: 10.3543 - kl: 0.0000e+00 - mmd: 12.9835 - val_loss: 50.4096 - val_recon: 10.1527 - val_kl: 0.0000e+00 - val_mmd: 44.7478
Epoch 3/30
87/87 [==============================] - 1s 7ms/step - loss: 17.2503 - recon: 7.7502 - kl: 0.0000e+00 - mmd: 9.4217 - val_loss: 114.0227 - val_recon: 14.4192 - val_kl: 0.0000e+00 - val_mmd: 97.5239
Epoch 4/30
87/87 [==============================] - 1s 7ms/step - loss: 13.6136 - recon: 6.4237 - kl: 0.0000e+00 - mmd: 7.1246 - val_loss: 65.2234 - val_recon: 6.9797 - val_kl: 0.0000e+00 - val_mmd: 58.4609
Epoch 5/30
87/87 [==============================] - 1s 7ms/step - loss: 11.6580 - recon: 5.8392 - kl: 0.0000e+00 - mmd: 5.9652 - val_loss: 93.6862 - val_recon: 6.7

In [26]:
cp_vae.vae

In [27]:
# Save training performance
history_df = pd.DataFrame(cp_vae.vae.history.history)
history_df

,loss,recon,kl,mmd,val_loss,val_recon,val_kl,val_mmd
0,53.256985,33.495697,0.0,20.150007,55.295570,10.631577,0.0,42.863033
1,23.473732,10.354336,0.0,12.983544,50.409554,10.152746,0.0,44.747761
2,17.250278,7.750245,0.0,9.421741,114.022728,14.419161,0.0,97.523895
3,13.613574,6.423739,0.0,7.124564,65.223373,6.979677,0.0,58.460938
4,11.657954,5.839181,0.0,5.965222,93.686180,6.759391,0.0,86.974808
5,10.168350,5.576635,0.0,4.696644,42.958366,5.722135,0.0,34.796650
6,9.831089,5.465136,0.0,4.591119,17.252010,5.386589,0.0,13.400815
7,9.650624,5.411336,0.0,4.306530,30.674387,5.312686,0.0,26.383856
8,9.534864,5.379193,0.0,4.270781,21.932907,5.294547,0.0,17.708151
9,9.359639,5.363744,0.0,4.061123,54.098064,5.307471,0.0,53.501514


In [28]:
encoder = cp_vae.encoder_block["encoder"]
decoder = cp_vae.decoder_block["decoder"]
encoder.save("models/level5EncoderShuffled_mmd")
decoder.save("models/level5DecoderShuffled_mmd")

INFO:tensorflow:Assets written to: models/level5EncoderShuffled_mmd/assets
INFO:tensorflow:Assets written to: models/level5DecoderShuffled_mmd/assets


In [29]:
# history_df.to_csv('level5_training_mmd_random.csv')

In [ ]:
# original_training_data  = pd.read_csv('level5_training_mmd.csv')

In [ ]:
# plt.figure(figsize=(7, 5), dpi = 400)
# plt.plot(original_training_data["loss"], label="Training data")
# plt.plot(original_training_data["val_loss"], label="Validation data")
# plt.plot(history_df["loss"], label="Shuffled training data")
# plt.plot(history_df["val_loss"], label="Shuffled validation data")
# # plt.title("Loss for VAE training on Cell Painting Level 5 data")
# plt.ylabel("MSE + MMD")
# plt.xlabel("No. Epoch")
# plt.legend()
# plt.show()